In [1]:
%run /utils/common_functions

In [2]:
account_name = raw_adls_path.split('@')[1].split('.')[0]
account_fqdn  = account_name + ".dfs.core.windows.net"
print(account_name)
print(account_fqdn)

In [3]:
raw_container = 'raw'

# Define Source path

In [4]:
centric_base = "/Centric"

plm_projects_path        = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/PLM_Projects.parquet"
ed_season_path           = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_SEASON.parquet"
ed_country_path          = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_COUNTRY.parquet"
ed_user_path             = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_USER.parquet"
ed_supplier_path         = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_SUPPLIER.parquet"
ed_factory_path          = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_FACTORY.parquet"
ed_supplier_factory_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_SUPPLIER_FACTORY.parquet"
ed_lookup_item_path      = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_LOOKUP_ITEM.parquet"


In [5]:
ed_colorway_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_COLORWAY.parquet"
ed_style_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_STYLE.parquet"
ed_color_product_source_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_COLOR_PRODUCT_SOURCE.parquet"
ed_product_source_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_PRODUCT_SOURCE.parquet"

ed_enum_value_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_ENUM_VALUE.parquet"
ed_size_range_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_SIZE_RANGE.parquet"
ed_material_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_MATERIAL.parquet"
ed_theme_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_THEME.parquet"
ed_color_specification_path = f"abfss://{raw_container}@{account_name}.dfs.core.windows.net{centric_base}/ED_COLOR_SPECIFICATION.parquet"




In [6]:
print(f"centric_base               = {centric_base}")

print(f"plm_projects_path          = {plm_projects_path}")
print(f"ed_season_path             = {ed_season_path}")
print(f"ed_country_path            = {ed_country_path}")
print(f"ed_user_path               = {ed_user_path}")
print(f"ed_supplier_path           = {ed_supplier_path}")
print(f"ed_factory_path            = {ed_factory_path}")
print(f"ed_supplier_factory_path   = {ed_supplier_factory_path}")
print(f"ed_lookup_item_path        = {ed_lookup_item_path}")


# Read Data

In [7]:
#df_plm_projects = spark.read.parquet(plm_projects_path)

df_ed_season = spark.read.parquet(ed_season_path)

df_ed_country = spark.read.parquet(ed_country_path)

df_ed_user = spark.read.parquet(ed_user_path)

df_ed_supplier = spark.read.parquet(ed_supplier_path)

df_ed_factory = spark.read.parquet(ed_factory_path)

df_ed_supplier_factory = spark.read.parquet(ed_supplier_factory_path)

df_ed_lookup_item = spark.read.parquet(ed_lookup_item_path)


In [8]:
df_ed_colorway = spark.read.parquet(ed_colorway_path)

df_ed_style = spark.read.parquet(ed_style_path)

df_ed_color_product_source = spark.read.parquet(ed_color_product_source_path)

df_ed_product_source = spark.read.parquet(ed_product_source_path)

df_ed_enum_value = spark.read.parquet(ed_enum_value_path)

df_ed_size_range = spark.read.parquet(ed_size_range_path)

df_ed_material = spark.read.parquet(ed_material_path)

df_ed_theme = spark.read.parquet(ed_theme_path)

df_ed_color_specification = spark.read.parquet(ed_color_specification_path)


In [9]:
from pyspark.sql import functions as F

# ------------------------------------------------------------------------------
# Helper expressions to mimic SQL CHARINDEX/SUBSTRING patterns on "A:B:C" strings
# ------------------------------------------------------------------------------

def after_first_col(c):
    # SQL: SUBSTRING(x, CHARINDEX(':', x) + 1, LEN(x))
    return F.regexp_extract(F.coalesce(c, F.lit("")), r"^[^:]*:(.*)$", 1)

def after_second_col(c):
    # SQL: SUBSTRING(x, CHARINDEX(':', x, CHARINDEX(':', x) + 1) + 1, LEN(x))
    return F.regexp_extract(F.coalesce(c, F.lit("")), r"^[^:]*:[^:]*:(.*)$", 1)

def between_first_and_second_col(c):
    # SQL: SUBSTRING(x, first_colon+1, second_colon-first_colon-1)
    return F.regexp_extract(F.coalesce(c, F.lit("")), r"^[^:]*:([^:]*):", 1)

def after_second_col_to_end(c):
    # SQL: SUBSTRING(x, second_colon+1, LEN(x))
    return F.regexp_extract(F.coalesce(c, F.lit("")), r"^[^:]*:[^:]*:(.*)$", 1)


# ------------------------------------------------------------------------------
# CTE: factory_info_cte
# ------------------------------------------------------------------------------

factory_info_cte = (
    df_ed_colorway.alias("a")
    .join(df_ed_color_product_source.alias("b"), F.col("a.ID") == F.col("b.COLORWAY"), "left")
    .join(df_ed_product_source.alias("c"), F.col("b.THE_PARENT_ID") == F.col("c.ID"), "left")
    .join(df_ed_supplier_factory.alias("d"), F.col("c.SUPPLIER_FACTORY") == F.col("d.ID"), "left")
    .join(df_ed_supplier.alias("f"), F.col("d.THE_PARENT_ID") == F.col("f.ID"), "inner")
    .join(df_ed_factory.alias("e"), F.col("d.FACTORY") == F.col("e.ID"), "left")
    .join(df_ed_country.alias("h"), F.col("e.COUNTRY") == F.col("h.ID"), "left")
    .join(df_ed_season.alias("season"), F.col("a.PARENT_SEASON") == F.col("season.PARENT_SEASON"), "left")
    .select(
        F.col("a.WVW_STOCK_NUMBER").alias("Stock_Number"),
        F.col("season.NODE_NAME").alias("Season"),
        F.col("season.CODE").alias("Season_Code"),
        F.col("a.ACTIVE").alias("CWActive"),
        F.col("e.SUPPLIER_NUMBER").alias("FactoryID"),
        F.col("e.NODE_NAME").alias("Factory"),
        F.col("f.NODE_NAME").alias("Supplier"),
        F.col("h.NODE_NAME").alias("CountryOfOrigin"),
        F.col("h.CODE").alias("CountryOfOriginCode"),
        F.col("b.ACTIVE").alias("AllocationActive"),
        F.col("b.WVW_FIN_SPEC_LOCK_UPL_APP_BY").alias("SpecLockBy"),
        F.col("b.WVW_FIN_SPEC_LOCKED_UPL_DATE").alias("SpecLockOn"),
        F.col("b.WVW_WBS_BUY_READY").alias("BuyReadyFlag"),
    )
    .dropDuplicates()
)

# ------------------------------------------------------------------------------
# Lookup/User CTEs (these are simple projections in SQL)
# ------------------------------------------------------------------------------

# raw.Centric_ED_LOOKUP_ITEM projections
lokup_node_name_WVW_PRODUCT_HIERARCHY_cte = df_ed_lookup_item.select("ID", "NODE_NAME")
lokup_node_name_WVW_SUB_FRANCHISE_cte     = df_ed_lookup_item.select("ID", "NODE_NAME")
lokup_node_name_WVW_CUSTOMER_cte          = df_ed_lookup_item.select("ID", "CODE", "NODE_NAME")
lokup_node_name_WVW_FRANCHISE_cte         = df_ed_lookup_item.select("ID", "CODE", "NODE_NAME")
lokup_node_name_WVW_PATTERN_NAME_cte      = df_ed_lookup_item.select("ID", "NODE_NAME", "WVW_PATTERN_DESCRIPTION")
lokup_node_name_WVW_GENDER_cte            = df_ed_lookup_item.select("ID", "NODE_NAME")
lokup_node_name_WVW_MATERIAL_FREIGHT_GROUP_cte = df_ed_lookup_item.select("ID", "NODE_NAME")
lokup_pictogram_cte = df_ed_lookup_item.select("ID", "WVW_LINING_DESCRIPTION", "WVW_OUTSOLE_DESCRIPTION", "WVW_UPPER_DESCRIPTION")

# raw.Centric_ED_USER projections
user_node_name_WVW_DESIGNER_cte         = df_ed_user.select("ID", "NODE_NAME")
user_node_name_WVW_COLORIST_cte         = df_ed_user.select("ID", "NODE_NAME")
user_node_name_WVW_DEVELOPER_cte        = df_ed_user.select("ID", "NODE_NAME")
user_node_name_WVW_SOURCING_DEVELOPER_cte = df_ed_user.select("ID", "NODE_NAME")
user_node_name_THE_CREATED_BY_cte       = df_ed_user.select("ID", "NODE_NAME")
user_node_name_WVW_ACCOUNTABLE_cte      = df_ed_user.select("ID", "NODE_NAME")
user_node_name_WVW_ACCOUNTABLE_cw_cte   = df_ed_user.select("ID", "NODE_NAME")
user_node_name_THE_CREATED_BY_CW_cte    = df_ed_user.select("ID", "NODE_NAME")

# NRF lookup CTE
lokup_nrf_cte = (
    df_ed_color_specification.alias("a")
    .join(df_ed_lookup_item.alias("b"), F.col("a.WVW_NRF_CODE") == F.col("b.ID"), "left")
    .select(
        F.col("a.ID").alias("ID"),
        F.col("a.WVW_NRF_CODE").alias("WVW_NRF_CODE"),
        F.col("b.WVW_SAP_CODE").alias("WVW_SAP_CODE"),
        F.col("b.NODE_NAME").alias("NODE_NAME"),
    )
    .dropDuplicates(["ID"])
)

# Icons CTEs (same filter repeated 4 times in SQL)
icons_base = df_ed_lookup_item.select("ID", "NODE_NAME").filter(F.col("NODE_NAME").like("icon-%"))
Icon1_cte = icons_base
Icon2_cte = icons_base
Icon3_cte = icons_base
Icon4_cte = icons_base


# ------------------------------------------------------------------------------
# Final SELECT (replicates [gold].[Centric_PLM_Projects])
# ------------------------------------------------------------------------------

cw = df_ed_colorway.alias("cw")
style = df_ed_style.alias("style")

season1 = df_ed_season.alias("season1")
season2 = df_ed_season.alias("season2")
season3 = df_ed_season.alias("season3")

df_plm_projects_snapshot = (
    cw
    .join(style, F.col("cw.THE_PARENT_ID") == F.col("style.ID"), "left")

    .join(lokup_node_name_WVW_PRODUCT_HIERARCHY_cte.alias("lkp_prod_hier"),
          F.col("style.WVW_PRODUCT_HIERARCHY") == F.col("lkp_prod_hier.ID"), "left")

    .join(lokup_node_name_WVW_SUB_FRANCHISE_cte.alias("lkp_sub_fran"),
          F.col("cw.WVW_SUB_FRANCHISE") == F.col("lkp_sub_fran.ID"), "left")

    .join(season1, F.col("season1.ID") == F.col("cw.WVW_ORIGINAL_ENDOF_LIFE"), "left")
    .join(season2, F.col("season2.ID") == F.col("cw.WVW_REVISED_ENDOF_LIFE"), "left")

    .join(season3, F.col("cw.PARENT_SEASON") == F.col("season3.PARENT_SEASON"), "left")

    .join(factory_info_cte.alias("factory_info"),
          (F.col("cw.WVW_STOCK_NUMBER") == F.col("factory_info.Stock_Number")) &
          (F.col("season3.CODE") == F.col("factory_info.Season_Code")) &
          (F.col("cw.ACTIVE") == F.col("factory_info.CWActive")),
          "left")

    .join(lokup_node_name_WVW_PATTERN_NAME_cte.alias("lkp_pattern"),
          F.col("style.WVW_PATTERN_NAME") == F.col("lkp_pattern.ID"), "left")

    .join(df_ed_enum_value.alias("enumval"),
          F.col("enumval.VALUE_001") == F.col("cw.WVW_PRODUCT_LINE"), "left")

    .join(lokup_node_name_WVW_CUSTOMER_cte.alias("lkp_customer"),
          F.col("cw.WVW_CUSTOMER") == F.col("lkp_customer.ID"), "left")

    .join(lokup_node_name_WVW_FRANCHISE_cte.alias("lkp_franchise"),
          F.col("cw.WVW_FRANCHISE") == F.col("lkp_franchise.ID"), "left")

    .join(lokup_node_name_WVW_GENDER_cte.alias("lkp_gender"),
          F.col("style.WVW_GENDER") == F.col("lkp_gender.ID"), "left")

    .join(lokup_node_name_WVW_MATERIAL_FREIGHT_GROUP_cte.alias("lkp_mfg"),
          F.col("style.WVW_MATERIAL_FREIGHT_GROUP") == F.col("lkp_mfg.ID"), "left")

    .join(user_node_name_WVW_DESIGNER_cte.alias("u_designer"),
          F.col("style.WVW_DESIGNER") == F.col("u_designer.ID"), "left")
    .join(user_node_name_WVW_COLORIST_cte.alias("u_colorist"),
          F.col("style.WVW_COLORIST") == F.col("u_colorist.ID"), "left")
    .join(user_node_name_WVW_DEVELOPER_cte.alias("u_developer"),
          F.col("style.WVW_DEVELOPER") == F.col("u_developer.ID"), "left")
    .join(user_node_name_WVW_SOURCING_DEVELOPER_cte.alias("u_sourcing_dev"),
          F.col("style.WVW_SOURCING_DEVELOPER") == F.col("u_sourcing_dev.ID"), "left")
    .join(user_node_name_THE_CREATED_BY_cte.alias("u_style_created_by"),
          F.col("style.THE_CREATED_BY") == F.col("u_style_created_by.ID"), "left")
    .join(user_node_name_WVW_ACCOUNTABLE_cte.alias("u_responsible"),
          F.col("style.WVW_ACCOUNTABLE") == F.col("u_responsible.ID"), "left")
    .join(user_node_name_WVW_ACCOUNTABLE_cw_cte.alias("u_accountable_cw"),
          F.col("cw.WVW_ACCOUNTABLE") == F.col("u_accountable_cw.ID"), "left")

    .join(df_ed_size_range.alias("size"),
          F.col("style.ACTUAL_SIZE_RANGE") == F.col("size.ID"), "left")

    .join(df_ed_material.alias("mat"),
          F.col("style.WVW_BOTTOMING_OUTSOLE_1") == F.col("mat.ID"), "left")

    .join(df_ed_color_specification.alias("colorspec"),
          F.col("cw.COLOR_SPECIFICATION") == F.col("colorspec.ID"), "left")

    .join(user_node_name_THE_CREATED_BY_CW_cte.alias("u_cw_created_by"),
          F.col("cw.THE_CREATED_BY") == F.col("u_cw_created_by.ID"), "left")

    .join(df_ed_theme.alias("theme"),
          F.col("style.THEME") == F.col("theme.ID"), "left")

    .join(lokup_nrf_cte.alias("lkp_nrf"),
          F.col("cw.COLOR_SPECIFICATION") == F.col("lkp_nrf.ID"), "left")

    .join(lokup_pictogram_cte.alias("lkp_picto"),
          F.col("cw.WVW_PICTOGRAM_LABEL_ID") == F.col("lkp_picto.ID"), "left")

    .join(Icon1_cte.alias("icon1"), F.col("style.SB_ICON_1") == F.col("icon1.ID"), "left")
    .join(Icon2_cte.alias("icon2"), F.col("style.SB_ICON_2") == F.col("icon2.ID"), "left")
    .join(Icon3_cte.alias("icon3"), F.col("style.SB_ICON_3") == F.col("icon3.ID"), "left")
    .join(Icon4_cte.alias("icon4"), F.col("style.SB_ICON_4") == F.col("icon4.ID"), "left")

    .select(
        # --- CW / Stock ---
        F.col("cw.WVW_STOCK_NUMBER").alias("StockNumber"),
        F.col("cw.WVW_MATERIAL_DESCRIPTION").alias("MaterialDescription"),

        # DivisionCode / Division from style.WVW_DIVISION_HIERARCHY
        between_first_and_second_col(F.col("style.WVW_DIVISION_HIERARCHY")).alias("DivisionCode"),
        after_second_col(F.col("style.WVW_DIVISION_HIERARCHY")).alias("Division"),

        # Lookups / attributes
        F.col("lkp_prod_hier.NODE_NAME").alias("ProductHierarchySeasonless"),
        F.col("lkp_sub_fran.NODE_NAME").alias("SubFranchiseSeasonless"),

        # Factory info
        F.col("factory_info.FactoryID").alias("FactoryID"),
        F.col("factory_info.Factory").alias("Factory"),
        F.col("factory_info.Supplier").alias("Supplier"),
        F.col("factory_info.CountryOfOriginCode").alias("CountryOfOriginCode"),
        F.col("factory_info.CountryOfOrigin").alias("CountryOfOrigin"),

        # Style / CW fields
        F.col("style.WVW_AGE").alias("AgeCode"),
        after_first_col(F.col("cw.WVW_STATUS")).alias("ColorwayStatus"),
        F.col("factory_info.AllocationActive").alias("AllocationActive"),
        F.col("factory_info.CWActive").alias("CWActive"),
        after_second_col(F.col("style.WVW_BASIC_MATERIAL")).alias("BasicMaterial"),
        F.col("style.WVW_MASTER_GRID").alias("MasterGrid"),
        F.col("cw.NODE_NAME").alias("DevelopmentColorwayName"),
        F.col("cw.WVW_COLORWAY_WORKING_NAME").alias("ColorwayWorkingName"),
        F.col("cw.ID").alias("ColorwayID"),

        # Color specification / NRF
        F.col("colorspec.NODE_NAME").alias("ColorSpecification"),
        F.col("lkp_nrf.WVW_SAP_CODE").alias("NRFCodeSAPCode"),
        F.col("lkp_nrf.NODE_NAME").alias("NRFCodeC8demo"),

        # More fields
        F.col("style.WVW_FISHAND_WILDLIFE").alias("FishAndWildlife"),
        F.col("enumval.DESCRIPTION").alias("GlobalStatus"),
        F.col("lkp_customer.CODE").alias("CustomerNumber"),
        F.col("lkp_customer.NODE_NAME").alias("Customer"),
        F.col("cw.WVW_PICTOGRAM_LABEL_ID").alias("PictogramLabelID"),
        F.col("lkp_picto.WVW_LINING_DESCRIPTION").alias("LiningDescription"),
        F.col("lkp_picto.WVW_OUTSOLE_DESCRIPTION").alias("OutsoleDescription"),
        F.col("style.WVW_CUTTING_PATTERN").alias("CuttingPattern"),
        F.col("lkp_franchise.NODE_NAME").alias("FranchiseSeasonless"),
        after_first_col(F.col("cw.WVW_CONSUMER_TERRITORY")).alias("ConsumerTerritory"),
        after_first_col(F.col("cw.WVW_STORY_TYPE")).alias("StoryType"),
        after_first_col(F.col("cw.WVW_PRODUCT_TYPE_CLASS")).alias("ProductTypeSpecification"),
        after_first_col(F.col("cw.WVW_PRODUCT_TIER")).alias("ProductTierSeasonless"),
        after_first_col(F.col("cw.WVW_LAUNCH_MONTH")).alias("LaunchMonth"),
        F.col("lkp_picto.WVW_UPPER_DESCRIPTION").alias("UpperDescription"),
        F.col("cw.WVW_COLORWAY_CODE").alias("ColorwayCode"),
        F.col("cw.WVW_SEASONAL_COLORWAY_CODE").alias("SeasonalColorwayCode"),
        F.col("u_accountable_cw.NODE_NAME").alias("AccountableSeasonless"),
        F.col("cw.THE_CREATED_AT").alias("CreatedOn"),
        F.col("u_cw_created_by.NODE_NAME").alias("CreatedBy"),
        F.col("style.WVW_SPEC_LOCK_INDICATOR").alias("SpecLockIndicator"),
        F.col("factory_info.SpecLockBy").alias("SpecLockBy"),
        F.col("factory_info.SpecLockOn").alias("SpecLockOn"),
        F.col("season1.NODE_NAME").alias("OriginalEndOfLife"),
        F.col("season2.NODE_NAME").alias("RevisedEndOfLife"),
        after_first_col(F.col("size.WVW_SHOE_WIDTH")).alias("Width"),
        F.col("factory_info.BuyReadyFlag").alias("BuyReadyFlag"),
        F.col("cw.WVW_INITIAL_SEASON").alias("InitialSeason"),
        F.col("size.DESCRIPTION").alias("Sizes"),

        # Season3 (by PARENT_SEASON)
        F.col("season3.CODE").alias("SeasonCode"),
        F.col("season3.NODE_NAME").alias("Season"),
        F.col("season3.SB_SAP_SEASON").alias("SAPSeason"),
        F.col("season3.WVW_SEASON_QUARTER").alias("SeasonQuarter"),

        # Style core
        after_first_col(F.col("style.WVW_STYLE_CODE")).alias("C8StyleCode"),
        F.col("style.CODE").alias("StyleCode"),
        F.col("style.WVW_SEASONAL_STYLE_CODE").alias("SeasonalStyleCode"),
        F.col("style.WVW_STYLE_NAME").alias("StyleWorkingName"),
        F.col("style.WVW_STYLE_NAME").alias("FinalPatternNameSeasonless"),
        F.col("lkp_pattern.WVW_PATTERN_DESCRIPTION").alias("PatternDescription"),
        after_first_col(F.col("style.WVW_STATUS")).alias("StyleStatus"),
        after_first_col(F.col("style.WVW_DEVELOPMENT_TYPE")).alias("DevelopmentType"),
        after_first_col(F.col("cw.WVW_DEVELOPMENT_TYPE")).alias("ColorwayDevelopmentType"),
        F.col("lkp_gender.NODE_NAME").alias("Gender"),
        F.col("style.WVW_LAST").alias("Last"),
        F.col("style.WVW_LAST_NAME").alias("LastName"),
        F.col("style.WVW_NEW_LAST").alias("NewLast"),
        after_first_col(F.col("style.WVW_CONSTRUCTION")).alias("ConstructionSeasonless"),
        F.col("lkp_mfg.NODE_NAME").alias("MaterialFreightGroupSeasonless"),
        after_second_col(F.col("style.WVW_BRAND")).alias("Brand"),

        # Material / midsole
        F.col("mat.NODE_NAME").alias("BottomingOutsole1"),
        F.col("mat.WVW_TS_MIDSOLE_1").alias("MidsoleName"),

        # Measurements / flags
        F.col("style.WVW_FOREFOOT_HEIGHT").alias("ForefootHeightSeasonless"),
        F.col("style.WVW_SHAFT_HEIGHT").alias("ShaftHeight"),
        F.col("style.WVW_SHAFT_CIRCUMFERENCE").alias("ShaftCircumference"),
        after_first_col(F.col("style.WVW_DROP")).alias("HeelDropSeasonless"),
        F.col("style.WVW_SAMPLE_PAIR_WEIGHT_KG").alias("SamplePairWeightKg"),
        F.col("style.WVW_HEEL_HEIGHT").alias("HeelDrop"),
        F.col("style.WVW_BOOTIE").alias("BootieSeasonless"),
        F.col("style.WVW_VIBRAM").alias("Vibram"),
        F.col("style.WVW_WATERPROOF").alias("WaterproofSeasonless"),
        F.col("style.WVW_NEW_KNIVES").alias("NewKnives"),

        # Users
        F.col("u_designer.NODE_NAME").alias("Designer"),
        F.col("style.WVW_CONTRACT_DESIGN").alias("ContractDesigner"),
        F.col("u_colorist.NODE_NAME").alias("Colorist"),
        F.col("u_developer.NODE_NAME").alias("Developer"),
        F.col("u_sourcing_dev.NODE_NAME").alias("SourcingDeveloper"),

        # Created / Responsible
        F.col("style.WVW_DEV_REF_STYLE").alias("DevReferenceStyle"),
        F.col("style.THE_CREATED_AT").alias("Created"),
        F.col("u_style_created_by.NODE_NAME").alias("StyleCreatedBy"),
        F.col("style.WVW_USA_TARGET_DUTY_FREE").alias("USADutyFree"),
        F.col("u_responsible.NODE_NAME").alias("Responsible"),

        after_first_col(F.col("style.WVW_SEAM_SEALED_1")).alias("SeamSealedSeasonless"),
        F.col("style.WVW_PLANNING_CATEGORY").alias("PlanningCategory"),
        after_first_col(F.col("style.WVW_SEASONAL_PROGRAM")).alias("SeasonalProgramSeasonless"),
        F.col("style.WVW_APPROVAL_NUMBER").alias("ApprovalNumber"),
        F.col("mat.DIAMETER").alias("Diameter"),
        F.col("style.WVW_TARGET_NUMBEROF_COLORS").alias("TargetNumberOfColors"),
        F.col("style.WVW_TARGET_PURCHASE_PRICE").alias("TargetPurchasePrice"),
        F.col("style.WVW_RETAIL_PRICE_MSRP").alias("RetailPriceMSRP"),
        F.col("style.WVW_WHOLESALE_PRICE").alias("WholesalePrice"),
        F.col("style.WVW_LABEL_PATTERN_1").alias("LabelPattern1"),
        F.col("style.WVW_LABEL_PATTERN_2").alias("LabelPattern2"),
        F.col("style.WVW_USA_TARGET_DUTY_PERCENT").alias("USATargetDutyPct"),
        F.col("style.WVW_CANADA_TARGET_DUTY_PERCENTAGE").alias("CanadaTargetDutyPct"),
        F.col("style.WVW_EUROPE_TARGET_DUTY_PERCENTAGE").alias("EuropeTargetDutyPct"),
        F.col("style.WVW_UK_TARGET_DUTY_PERCENTAGE").alias("UKTargetDutyPct"),
        after_first_col(F.col("style.WVW_ACTIVITY_TYPE")).alias("ActivityTypeSeasonless"),

        # Theme
        F.col("theme.NODE_NAME").alias("Theme"),
        after_first_col(F.col("style.WVW_PRODUCT_POSITIONING")).alias("BriefPositioning"),
        F.col("cw.WVW_DISTRIBUTION_CHANNEL").alias("WVW_DISTRIBUTION_CHANNEL"),
        F.col("theme.THE_CREATED_AT").alias("BRCreatedOn"),
        F.col("theme.THE_CREATED_BY").alias("BRCreatedBy"),
        after_first_col(F.col("theme.WVW_FIELD_TESTING")).alias("FieldTesting"),
        F.col("theme.DESCRIPTION").alias("Description"),
        after_first_col(F.col("theme.WVW_LAUNCH_MONTH")).alias("PlannedLaunchMonth"),
        after_second_col(F.col("theme.WVW_DIVISION")).alias("BRDivision"),
        after_first_col(F.col("theme.WVW_DEVELOPMENT_TYPE")).alias("ThemeDevelopmentType"),
        F.col("theme.WVW_PACKAGING").alias("PlannedPackaging"),
        F.col("theme.WVW_FACTORY").alias("PlannedFactory"),
        after_first_col(F.col("theme.WVW_BRIEF_TYPE")).alias("BriefType"),
        after_first_col(F.col("theme.WVW_SEGMENTATION")).alias("Consumer"),

        # Extra CW
        F.col("cw.WVW_SAP_PATTERN_NAME").alias("PRMaterialDesc"),
        after_first_col(F.col("cw.WVW_TOE_COMPONENT")).alias("ToeComponent"),
        F.col("cw.SB_RETAIL_SEASON").alias("SBRetailSeason"),

        # Icons
        F.col("icon1.NODE_NAME").alias("Icon1"),
        F.col("icon2.NODE_NAME").alias("Icon2"),
        F.col("icon3.NODE_NAME").alias("Icon3"),
        F.col("icon4.NODE_NAME").alias("Icon4"),

        # MSRP additions
        F.col("style.SB_MSRP_GBP").alias("SB_MSRP_GBP"),
        F.col("style.SB_MSRP_USD").alias("SB_MSRP_USD"),
        F.col("style.SB_MSRP_EUR").alias("SB_MSRP_EUR"),
    )
)

# df_plm_projects_snapshot is now the PySpark equivalent of [gold].[Centric_PLM_Projects]


In [12]:
df_plm_projects_snapshot.columns

In [11]:
display(df_plm_projects_snapshot)

# Write Data

In [14]:
gold_container = 'gold'
target_folder = 'Centric/PLM_Projects_Report3'
plm_projects_delta_table_path = f"abfss://{gold_container}@{account_name}.dfs.core.windows.net/{target_folder}"
print(plm_projects_delta_table_path)

In [15]:
df_plm_projects_snapshot.write.format("delta").mode("overwrite").option("low_memory", "false").save(plm_projects_delta_table_path)